In [1]:
from pprint import pprint

import parser
import ast
import inspect

import numpy as np
import pandas as pd

In [2]:
def add(x, y=0):
    return x, y

In [19]:
# Label
add(
    1,
    y=2+3,
)

(1, 5)

In [20]:
text = In[19]
print(text)

# Label
add(
    1,
    y=2+3,
)


In [22]:
## Split text into lines
lines = text.split('\n')
print('lines: {}\n'.format(lines))

# Get comment lines
comment_lines = [l.strip() for l in lines if l.startswith('#')]
print('comment_lines: {}\n'.format(comment_lines))

# Get label from the 1st comment line
label = ''.join(comment_lines[0].split('#')[1:]).strip()
print('label: {}\n'.format(label))

# Get code lines
code_lines = [l.strip() for l in lines if not l.startswith('#')]
print('comment_lines: {}\n'.format(comment_lines))

# Make AST
m = ast.parse(text)
b = m.body[0]

# Get returns
returns = []
if isinstance(b, ast.Assign):
    peek = b.targets[0]
    if isinstance(peek, ast.Tuple):
        targets = peek.elts
    elif isinstance(peek, ast.Name):
        targets = b.targets
    for t in targets:
        returns.append({
            'label': '{} Label'.format(t.id),
            'description': 'Description.',
            'value': t.id,
        })
print('returns:')
pprint(returns)
print()

# Get function name
function_name = b.value.func.id
print('function_name: {}\n'.format(function_name))

# Get args and kwargs
args = []
kwargs = {}
for a in [l for l in code_lines if not (l.endswith('(') or l.startswith(')'))]:
    if '=' in a:  # kwarg
        k, v = a.split('=')
        if v.endswith(','):
            v = v[:-1]
        kwargs[k] = v
    else:  # arg
        if a.endswith(','):
            a = a[:-1]
        args.append(a)
print('args: {}\n'.format(args))
print('kwargs: {}\n'.format(kwargs))

# Get function's signature
s = inspect.signature(add)
print('inspected parameters:')
for k, v in s.parameters.items():
    print('\t{}: {}'.format(k, v))

# Get required args
required_args = [{
    'label': '{} Label'.format(n),
    'description': 'Description.',
    'name': n,
    'value': v,
} for n, v in zip([v.name for v in s.parameters.values() if v.default == inspect._empty], args)]
print('required_args:')
pprint(required_args)
print()

# Get optional args
optional_args = [{
    'label': '{} Label'.format(n),
    'description': 'Description.',
    'name': n,
    'value': v,
} for n, v in kwargs.items()]
print('optional_args:')
pprint(optional_args)
print()

# Get module name
module_name = eval('{}.__module__'.format(function_name))
print('module_name: {}\n'.format(module_name))

# Get module path
if module_name == '__main__':  # Function is defined within this Notebook
    module_path = ''
else:  # Function is imported from a module
    module_path = eval(
        '{}.__globals__.get(\'__file__\')'.format(function_name)
    ).split(module_name.replace('.', '/'))[0]
print('module_path: {}\n'.format(module_path))

# Make a task
task = {
    label: {
        'description': 'Description.',
        'library_path': module_path,
        'library_name': module_name,
        'function_name': function_name,
        'required_args': required_args,
        'default_args': [],
        'optional_args': optional_args,
        'returns': returns
    }
}
print('task:')
pprint(task)
print()

# # Register this task
# self._update_tasks(task)

# return task

lines: ['# Label', 'add(', '    1,', '    y=2+3,', ')']

comment_lines: ['# Label']

label: Label

comment_lines: ['# Label']

returns:
[]

function_name: add

args: ['1']

kwargs: {'y': '2+3'}

inspected parameters:
	x: x
	y: y=0
required_args:
[{'description': 'Description.', 'label': 'x Label', 'name': 'x', 'value': '1'}]

optional_args:
[{'description': 'Description.',
  'label': 'y Label',
  'name': 'y',
  'value': '2+3'}]

module_name: __main__

module_path: 

task:
{'Label': {'default_args': [],
           'description': 'Description.',
           'function_name': 'add',
           'library_name': '__main__',
           'library_path': '',
           'optional_args': [{'description': 'Description.',
                              'label': 'y Label',
                              'name': 'y',
                              'value': '2+3'}],
           'required_args': [{'description': 'Description.',
                              'label': 'x Label',
                              'n